# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("World_Weather_Analysis.csv")

# Display sample data
city_data_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,whitehorse,60.7161,-135.0538,281.58,89,75,5.66,CA,1697679176
1,1,frenda,35.0654,1.0495,289.55,77,39,1.70,DZ,1697679696
2,2,qaqortoq,60.7167,-46.0333,271.76,58,52,2.12,GL,1697679199
3,3,papatowai,-46.5619,169.4708,290.03,49,77,6.34,NZ,1697679171
4,4,hermanus,-34.4187,19.2345,289.33,85,37,7.90,ZA,1697679169


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
all_city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha = 0.5,
    color = "Country")

# Display the map
all_city_map

#

C:\Users\leahm\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
min_temp = 45
max_temp = 75

# Filter cities that meet the temperature criteria
filtered_cities_df = city_data_df[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)]

# Drop rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display the resulting DataFrame
filtered_cities_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Create an empty DataFrame to store hotel data
hotel_data_df = pd.DataFrame(city_data_df)

# Add an empty "Hotel Name" column to the DataFrame
hotel_data_df['Hotel Name'] = ''

# Display sample data
print(hotel_data_df.head())  

   Unnamed: 0        City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
0           0  whitehorse  60.7161 -135.0538    281.58        89          75   
1           1      frenda  35.0654    1.0495    289.55        77          39   
2           2    qaqortoq  60.7167  -46.0333    271.76        58          52   
3           3   papatowai -46.5619  169.4708    290.03        49          77   
4           4    hermanus -34.4187   19.2345    289.33        85          37   

   Wind Speed Country        Date Hotel Name  
0        5.66      CA  1697679176             
1        1.70      DZ  1697679696             
2        2.12      GL  1697679199             
3        6.34      NZ  1697679171             
4        7.90      ZA  1697679169             


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_data_df DataFrame
for index, row in hotel_data_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_data_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_data_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_data_df.loc[index, 'City']} - nearest hotel: {hotel_data_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_data_df

















Starting hotel search
whitehorse - nearest hotel: Town & Mountain Hotel
frenda - nearest hotel: No hotel found
qaqortoq - nearest hotel: Hotel Qaqortoq
papatowai - nearest hotel: No hotel found
hermanus - nearest hotel: Aloe guest house
iqaluit - nearest hotel: Frobisher Inn
port-aux-francais - nearest hotel: Keravel
yellowknife - nearest hotel: No hotel found
touba - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
sakakah - nearest hotel: فندق النزل
san ignacio de velasco - nearest hotel: Hotel Caparu
guiratinga - nearest hotel: No hotel found
new plymouth - nearest hotel: Carrington Motel
grytviken - nearest hotel: No hotel found
waiuku - nearest hotel: The Kentish Hotel
karachi - nearest hotel: Quetta Agha Hotel
waitangi - nearest hotel: Hotel Chathams
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
talnakh - nearest hotel: Талнах
abepura - nearest hotel: Yoyo
tiksi - nearest hotel: Арктика
bethel - nearest 

cooma - nearest hotel: Royal Hotel
praia da vitoria - nearest hotel: Salles
geraldton - nearest hotel: Ocean Centre Hotel
tosontsengel - nearest hotel: Алтай
nadym - nearest hotel: Янтарная
malabrigo - nearest hotel: No hotel found
tanjung pandan - nearest hotel: Billiton Hotel & Club
bamboo flat - nearest hotel: Hotel Megapode Nest
kununurra - nearest hotel: Kununurra Country Club Resort
posto fiscal rolim de moura - nearest hotel: No hotel found
ziro - nearest hotel: No hotel found
daru - nearest hotel: Kuki
kupang - nearest hotel: HOTEL SALUNGA
aasiaat - nearest hotel: SØMA
dzialdowo - nearest hotel: Varia
jining - nearest hotel: No hotel found
ostrow mazowiecka - nearest hotel: Zajazd "Skalny"
pangai - nearest hotel: Billy's Place
zhezqazghan - nearest hotel: No hotel found
northam - nearest hotel: Durrant House Hotel
badger - nearest hotel: Hotel North Pole
severo-yeniseyskiy - nearest hotel: Актолик
homer - nearest hotel: Driftwood Inn & Homer Seaside Lodges
manokwari - nearest h

fukuecho - nearest hotel: 五島第一ホテル
fale old settlement - nearest hotel: No hotel found
luderitz - nearest hotel: Seaview Hotel Zum Sperrgebiet
gounou gaya - nearest hotel: No hotel found
hammerfest - nearest hotel: Thon Hotel Hammerfest
zeya - nearest hotel: Серебряный створ
meulaboh - nearest hotel: Hotel Meuligou
mountain home - nearest hotel: Best Western Foothills Inn
kalmunai - nearest hotel: Saji new food
colonia - nearest hotel: Wasserturm Hotel Cologne
vicuna - nearest hotel: No hotel found
shiqian - nearest hotel: No hotel found
zarand - nearest hotel: No hotel found
bignona - nearest hotel: No hotel found
jaguariaiva - nearest hotel: No hotel found
ambon - nearest hotel: Hotel Hero
lorengau - nearest hotel: Seeadler Bay Hotel
yulinshi - nearest hotel: 榆溪大酒店
tofol - nearest hotel: Nautilus Resort
ola - nearest hotel: No hotel found
acarau - nearest hotel: No hotel found
akureyri - nearest hotel: Hótel Norðurland
zierikzee - nearest hotel: Hotel Restaurant 't Poortuus
sechura - 

fisterra - nearest hotel: Hotel Temático do Banco Azul
bridgewater - nearest hotel: Comfort Inn
bulgan - nearest hotel: Bulgan Hotel
fairbanks - nearest hotel: Westmark Fairbanks Hotel
port augusta - nearest hotel: Hotel Commonwealth
beltran - nearest hotel: No hotel found
thames - nearest hotel: Kopu Station Hotel
roosevelt - nearest hotel: No hotel found
woodland park - nearest hotel: Holiday Inn Totowa Wayne
saint-louis - nearest hotel: No hotel found
levuka - nearest hotel: No hotel found
loreto - nearest hotel: No hotel found
quellon - nearest hotel: Aliwen
birjand - nearest hotel: مهمانسرای جهانگردی بیرجند
alofi - nearest hotel: Matavai Resort
pekan - nearest hotel: Ancasa Royale Hotel
yatou - nearest hotel: No hotel found


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,whitehorse,60.7161,-135.0538,281.58,89,75,5.66,CA,1697679176,Town & Mountain Hotel
1,1,frenda,35.0654,1.0495,289.55,77,39,1.70,DZ,1697679696,No hotel found
2,2,qaqortoq,60.7167,-46.0333,271.76,58,52,2.12,GL,1697679199,Hotel Qaqortoq
3,3,papatowai,-46.5619,169.4708,290.03,49,77,6.34,NZ,1697679171,No hotel found
4,4,hermanus,-34.4187,19.2345,289.33,85,37,7.90,ZA,1697679169,Aloe guest house
...,...,...,...,...,...,...,...,...,...,...,...
585,585,quellon,-43.1167,-73.6167,281.88,92,93,2.29,CL,1697679209,Aliwen
586,586,birjand,32.8663,59.2211,282.41,43,0,2.06,IR,1697679776,مهمانسرای جهانگردی بیرجند
587,587,alofi,-19.0595,-169.9187,300.09,50,40,8.75,NU,1697679235,Matavai Resort
588,588,pekan,3.4836,103.3996,299.21,74,89,2.12,MY,1697679776,Ancasa Royale Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
hotel_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

hotel_map



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Country,Hotel Name)